# Google Colab Cheatsheet

1. Free service provided by google to help students/scholars and researchers learn and develop machine learning models.
2. Go to https://colab.research.google.com and create a new python 3 notebook.
3. They also provide shared GPUs. To use them, 
    1. go to Runtime option
    2. from there, select change Runtime type
    3. change Hardware accelerator from None to GPU. 
4. Usually you get a shared GPU allocated with your VM machine. Very rarely, it may also happen that you don't get a GPU because all the GPUs already being used by other users.
5. When you login for the 1st time, google allocates you a virtual machine (VM) for next 12 hours. 
6. Configuration of VM allocated are:
    1. 12 GB RAM
    2. 33 GB hardisk space
    3. 2vCPU @ 2.2GHz
    4. Maximum continous usage duration is 12 hours. 
7. If you stay disconnected from google colab for more than 90 minutes without any code being executed, google will release your VM and all the data you stored in hardisk will be lost.
8. What happens in background ?
    1. When you connect to google colab, VM gets allocated and on the same VM, your notebook gets connected to a python kernel running on the same machine.
    2. Whenver you run a block of code/cell, it is pushed to a queue. The kernel to which notebook is connected fetches code to execute from the queue one by one and executes it.
    3. It may happen that you are connected to your notebook and suddenly your internet connection breaks, in that case the kernel doesn't die or shuts down. It stays as it is. You may also reconnect to the same kernel on clicking the Reconnect button on the top right of notebook.
    4. It may happen that you have executed a block/cell and while waiting for its execution to get completed, you are disconnected from kernel due to internet connection issues; in that case your block of code would get executed completely on the kernel. Although you may not see the output of the code, but all the variables that were used in the cell would maintain their state after code execution. You can confirm this by reconnecting to the same kernel by clicking on Reconnect button and then checking values of the variables used during execution of previous cells.
    5. On clicking Reconnect button, it may happen that it shows reconnecting for few seconds/minutes. This may happen because:
        1. You are still not connected to internet.
        2. You have pushed many code cells to the execution queue and kernel is still processing them. You'll get connected to same kernel only after kernel has completed execution of all the blocks in the queue.
9. How to use google colab for training on big datasets?
    1. You can download data from links using bash commands that you can run on python kernel.
    2. To run bash command on python notebook use exclamation symbol (!).
    3. After it you can write any linux command that you wish (except those that need sudo priviledges; reasons are obvious)
    4. To dowload files, you can use wget command. For example, to download a compressed or zip file, use its public url as:
        ``` 
        ! wget "public_url_of_file"
        ```
    5. Usually big datasets are uploaded and available in compressed format like tar or zip. You need to extract them. You can extract using unzip or tar command. For example, to extract and unzip a zip file: 
        ``` 
        ! unzip file_name.zip
        ```
    6. Although most of the common packages are already installed but in case you wish to install a package of your choice, you can also use pip install commands like:
        ```
        ! pip install package-name
        ```
    7. After installing a pip, you may need to restart the runtime (click on runtime option and then click restart runtime option) inorder for kernel to detect the newly installed package.
    8. Usually training a model on big datasets needs 1-2 days for 20-30 epochs (depending on the complexity of modee, size of dataset and no. of epochs). But we only get 12 hours of continous usage for one VM.
    9. To tackle this issue, you can use checkpoints. Do not pass no. of epochs to fit method, just pass it no of epochs as 1 and do the same task in a loop. After each epoch save the model checkpoint adn upload it to any persistent remote cloud storage.
    10. Using AWS S3 or digital ocean spaces can  be handy. To connect to those remote object storage services, we can use a package 'boto3'. To install this package use ! pip install boto3 and then import it using import boto3 command.
    11. Let us say you have saved all the checkpoints in a directory called model, then first use tar command to convert folder in a single tar file and then upload a single tar file to storage:
        Sample code:
        ``` 
        ! tar -czvf models.tar.gz models
        ! pip install boto3
        import boto3
        session = boto3.Session(aws_access_key_id="Put AWS Key ID here " , aws_secret_access_key = " Put secret key here" )
        s3 = session.resource('s3',endpoint_url="you can configure this endpoint for using digital ocean spaces ",region_name=" set region name of bucket")
        bucket = s3.Bucket("Bucekt Name")
        with open('models name.tar.gz', 'rb') as data:
            bucket.upload_fileobj(data, 'models.tar.gz')
        object_acl = s3.ObjectAcl('Bucket name','models.tar.gz')
        response = object_acl.put(ACL='public-read') # to make the file public so that you can download it using wget commands whenever you train next time
        ```
    12. Sample code for training model:
    ``` 
    no_of_epochs = 20
    for i in no_of_epochs:
            model.fit(X,Y,epochs=1)
            model.save("models/" + i + ".h5") # this will save ith model in models directory with filename as i.h5 where i would be an integer
            # here you can put code to compress model folder after each iteration and upload it.
    ```
    13. Using above code, let us say each iteration takes 2 hours to complete and you are left with 8-9 hours of usage, then hopefully 3-4 epochs would have completed and they gets uploaded to bucket before google colab destroys the VM and kernel on which code is being executed.
    14. To continue training the model, you'll need to download and preprocess data again on new VM and then download checkpoints from bucket, extract them and use load_model package from keras to load checkpoint as model in memory. 
    15. You can also preprocess data and store them locally first, then compress it and upload to bucket in case preprocessing of data is also time consuming step. 
    16. While using big datasets, you may not be able to load complete dataset in  RAM while fitting as size of RAM is 12GB only whereas matrix multiplication of such large datsets may exhaust all the RAM. In these cases, with keras, write a data_generator method that loads a batch from hardisk and provide them to fit method. This is known as progressive loading.